In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow import keras
import keras.layers as L
from keras.datasets import cifar10
from keras.applications import DenseNet121
from keras.applications import VGG16

In [4]:
(x_train, y_train), (x_test, y_test)=tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 272s 2us/step


In [13]:
def create_exemplar_set(size, images, trained_model):
    #'images' has images all of class=y
    for i in images:
        exemplars.append(calc_feature_vector(i, trained_model, layer_index))
    chosen_exemplars=exemplars[:size]
    mean_exemplar=sum(exemplars)/len(images)
    return chosen_exemplars, mean_exemplar

In [14]:
def create_custom_loss():
    gan_vs_nogan=keras.losses.BinaryCrossentropy(label_smoothing=0.1)
    type_of_gan=keras.losses.CategoricalCrossentropy(label_smoothing=0.1)
    combined_loss=tf.add(gan_vs_nogan, type_of_gan)
    return combined_loss

In [6]:
def create_model(n_classes, input_dim):
    '''densenet=DenseNet121(include_top=False, 
                         weights='imagenet', 
                         input_shape=(input_dim, input_dim,3), 
                         classes=n_classes)
    '''
    model=keras.Sequential(
    [
        L.Conv2D(64, kernel_size=3, padding='same', strides=2, input_shape=(input_dim, input_dim,3)),
        L.LeakyReLU(0.2),
        L.Conv2D(32, kernel_size=3, padding='same', strides=2),
        L.LeakyReLU(0.2),
        L.Conv2D(16, kernel_size=3, padding='same', strides=2),
        L.LeakyReLU(0.2),
        L.Flatten(),
        L.Dense(n_classes),
        L.Softmax()
    ]
    )
    return model

In [8]:
def train_model(model, n_epochs, x_train, y_train):
    model.compile(optimizer=keras.optimizers.Adam(0.02), 
                  loss=keras.losses.CategoricalCrossentropy, 
                  metrics=['accuracy'])
    model.fit(x=x_train, 
              y=y_train, 
              epochs=n_epochs)
    return model

In [12]:
def calc_feature_vector(image, model, layer_index):
    #extract feature vector from a layer of ALREADY-TRAINED model
    extractor=keras.Model(model.inputs(), [model.layers[layer_index].output])
    vector=extractor(image)
    #divide vector by norm
    return vector

In [7]:
model=create_model(10, 224)
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 112, 112, 64)      1792      
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 56, 56, 32)        18464     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 56, 56, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 16)        4624      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 28, 28, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 12544)             0